In [2]:
import matplotlib, cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import math
import os
import itertools
import random
%matplotlib inline

In [3]:
task_2_train = "Task2Dataset/Training/png/"
task_2_test = "Task2Dataset/TestWithoutRotations/images/"
task_2_test_annotations = "Task2Dataset/TestWithoutRotations/annotations/"

task_3_test_annotations = "Task3AdditionalTestDataset/annotations/"
task_3_test = "Task3AdditionalTestDataset/images/"

In [50]:
#refactored to class structure to simplify parameter tuning and testing of different matching algorithms, outlier rejection etc
class SIFT:
    #c_thresh=0.09 to get 0.03 used by lowe as used in paper
    def __init__(self, n_octaves=3, n_feat=0, c_thresh=0.04, e_thresh=10, lowe_thresh=0.7, ransac_thresh=5.0, sigma=1.6, k=2, min_matches=10, train_folder="", test_folder="", test_annotation_folder="", matcher=cv2.FlannBasedMatcher()):
        """
        Initialise parameters, by default set to OpenCV default values.
        Allows easy testing of parameters by simply calling SIFT(n_octaves=5), for example
        """
        self.n_octaves=n_octaves #num octaves used in computing DoG pyramid
        self.n_feat=n_feat #num features to retain when getting keypoints/descriptors
        self.c_thresh=c_thresh #contrast threshold used in SIFT feature detection
        self.e_thresh=e_thresh #edge threshold in SIFT feature detection
        self.lowe_thresh=lowe_thresh #threshold for lowe's ratio test
        self.ransac_thresh=ransac_thresh #threshold used in RANSAC outlier rejection
        self.sigma=sigma #dictates level of gaussian blur in DoG pyramid
        self.k=k #number of nearest neighbours obtained in knnmatch
        self.min_matches=min_matches #number of matches required to calculate homography - kinda redundant
        self.train_folder=train_folder
        self.test_folder=test_folder
        self.test_annotation_folder=test_annotation_folder
        self.test_image=""
        self.train_pts = {} # store keypoints of training images
        self.train_desc = {} # store descriptors of training images
        self.train_feat_count = {} #store number of features found for training image for scoring mechanism
        self.matcher = matcher #FlannBasedMatcher or BFMatcher
        self.matches = [] #store retained matches
        self.homography_matrix = "" #store homography matrix
        self.inlier_mask = "" #store inlier mask
        self.test_annotations = {} #store annotations for test images
        self.num_train_images=0


    def get_sift_train_features(self):
        """
        Gets SIFT keypoints and descriptors for all images in train folder
        Stored under dictionaries for easy access using image name
        """
        
        sift = cv2.SIFT_create(self.n_feat, self.n_octaves, self.c_thresh, self.e_thresh, self.sigma)

        for train_image in os.listdir(self.train_folder):
            
            image = cv2.imread(os.path.join(self.train_folder, train_image), 0)
            pt, desc = sift.detectAndCompute(image, None)
            self.train_pts[train_image] = pt
            self.train_desc[train_image] = desc
            self.train_feat_count[train_image] = len(pt)
            self.num_train_images+=1
        

    def get_annotations(self):
        """
        Get annotations for each test image
        """
        for annotation_file in os.listdir(self.test_annotation_folder):
            
            image_name = os.path.basename(annotation_file)
            annotation_file = open(os.path.join(self.test_annotation_folder, annotation_file), "r")
            
            annotation_lines = annotation_file.readlines()
            image_annot = []
            
            for line in annotation_lines:
                image_annot.append(line.split(",")[0])
            
            self.test_annotations[image_name] = image_annot
    

    def sift_matches_lowes(self, train_image_name, canvas):
        """
        Match SIFT features, using Lowe's Ratio Test as a means of outlier rejection
        """

        #Get test image features
        print(f"Test_image: {self.test_image}\n")
        print(f"Train_image: {train_image_name}\n")

        test_img = cv2.imread(self.test_image, 0)

        sift = cv2.SIFT_create(self.n_feat, self.n_octaves, self.c_thresh, self.e_thresh, self.sigma)
        test_points, test_descriptor = sift.detectAndCompute(test_img, None)
        
        #match features
        matches = self.matcher.knnMatch(self.train_desc[train_image_name], test_descriptor, self.k)

        #lowe's ratio test
        for one, two in matches:
            if one.distance < self.lowe_thresh * two.distance:
                self.matches.append([one])
        self.draw_boxes(train_image_name, test_points, test_descriptor)
        
        
    def draw_boxes(self, train_image_name, test_pts, test_desc):
        assert self.train_desc, "Descriptors empty, Run SIFT.get_sift_train_features()"
        assert self.train_pts, "Points empty, run SIFT.get_sift_train_features()"
        assert self.test_image, "No test image selected"
        canvas = cv2.imread("canvas_test.png")
        img1 = cv2.imread(os.path.join(self.train_folder, train_image_name))
        h, w = img1.shape[:2]
        
        img2 = cv2.imread(self.test_image)
        train_pts, train_desc = self.train_pts[train_image_name], self.train_desc[train_image_name]
        
        src_pts = np.float32([train_pts[m[0].queryIdx].pt for m in self.matches]).reshape(-1,1,2)        
        dst_pts = np.float32([test_pts[m[0].trainIdx].pt for m in self.matches]).reshape(-1,1,2)
        
        print(len(src_pts), len(dst_pts))
        if len(src_pts) < 4 or len(dst_pts) < 4:
            return
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, self.ransac_thresh)
        if M is None:
            return
        matchesMask = mask.ravel().tolist()

        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts, M)
        if np.any(dst < 0) or np.any(dst > 512):
            return
        print(f"Train_image shape ({h},{w})\n")
        print("Test_image shape: ({},{})\n".format(*img2.shape[:2]))
        print("Bounding box Vertices: ({})".format(np.int32(dst)))
        
        draw_params = dict(matchColor = (0,255,0), # draw matches in green color
               singlePointColor = None,
               matchesMask = matchesMask, # draw only inliers
               flags = 2)
        
        # img3 = cv2.drawMatches(img1, train_pts, img2, test_pts, self.matches, None, matchesMask=matchesMask)
        canvas = cv2.polylines(canvas, [np.int32(dst)], True, (0,0,255), 1, cv2.LINE_AA)
        cv2.imwrite("./canvas_test.png", canvas)
        # cv2.imshow("result", img3)
        # cv2.waitKey()


    def sift_matches_ransac(self, train_image_name):
        """
        Match SIFT features, obtain homography matrix and use RANSAC/inlier mask for outlier rejection
        """
        
        #Get test image features
        test_img = cv2.imread(self.test_image, 0)
        sift = cv2.SIFT_create(self.n_feat, self.n_octaves, self.c_thresh, self.e_thresh, self.sigma)
        test_points, test_descriptor = sift.detectAndCompute(test_img, None)
        #match features
        matches = self.matcher.knnMatch(self.train_desc[train_image_name], test_descriptor, self.k)

        if len(matches) > self.min_matches:
            #if sufficient matches to compute homography, calculate source/dest points
            src_pts = np.float32([self.train_pts[train_image_name][m[0].queryIdx].pt for m in matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([test_points[m[0].trainIdx].pt for m in matches]).reshape(-1, 1, 2)
            
            #get homography matrix + mask
            
            H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, self.ransac_thresh)
            self.homography_matrix = H
            inlier_mask = mask.ravel().tolist()
            #extract inlier matches
            self.matches = [matches[i] for i in range(len(matches)) if inlier_mask[i]]
        else:
            #cannot compute homography
            self.matches=[]


    def compare_scores_annot(self, scores, test_image):
        scores = dict(itertools.islice(scores.items(), len(self.test_annotations[test_image])))

        true_pos, false_pos, true_neg, false_neg = 0, 0, 0, 0
        print(self.test_annotations[test_image])
        print(scores)
        print("-----------------------------")
        for annotation in self.test_annotations[test_image]:

            if annotation in scores:
                true_pos+=1
            else:
                false_pos+=1
                false_neg+=1
        print(test_image, ":", true_pos, "/", (true_pos+false_pos))
        
        true_neg = self.num_train_images - false_neg - true_pos - false_pos

        return true_pos, false_pos, true_neg, false_neg
    
    def eval(self, lowes=False, normalise=False):
        """
        Get score for a specific test image
        Normalise == False: score is purely a ranking of the number of matches between train images and a test image
        Normalise == True: normalise number of matches by number of features extracted from train image
        If lowes=false, use ransac. Vice versa.
        """
        tp, fp, tn, fn = 0, 0, 0, 0
        for test_image in os.listdir(self.test_folder):
            self.test_image = os.path.join(self.test_folder, test_image)
            scores = {}
            
            canvas = cv2.imread(self.test_image)
            cv2.imwrite("./canvas_test.png", canvas)
            for image, desc in self.train_desc.items():
                # if image != "027-gas-station.png" and image != "016-house.png" and image !="011-trash.png":
                #     continue
                self.matches = []

                if lowes:
                    self.sift_matches_lowes(image, canvas)
                    # break
                else:
                    self.sift_matches_ransac(image)
                if normalise:
                    scores[image.split("-")[1].split(".")[0]] = len(self.matches) / self.train_feat_count[image]
                else:
                    scores[image.split("-")[1].split(".")[0]] = len(self.matches)
            break

            scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))
            test_image = test_image.split(".")[0]+".txt"
            true_pos, false_pos, true_neg, false_neg = self.compare_scores_annot(scores, test_image)
            tp+=true_pos
            tn+=true_neg
            fp+=false_pos
            fn+=false_neg
            
        # print("Accuracy:", ((tp+tn)/(tp+tn+fp+fn)))



In [51]:
sift = SIFT(train_folder=task_2_train, 
            test_folder=task_2_test, 
            test_annotation_folder=task_2_test_annotations, 
            matcher=cv2.BFMatcher_create(),
            lowe_thresh=0.8, min_matches=3, n_feat=1000, n_octaves=5, c_thresh=0.1)

sift.get_sift_train_features()
sift.eval(normalise=True, lowes=True)


Test_image: Task2Dataset/TestWithoutRotations/images/test_image_1.png

Train_image: 001-lighthouse.png

16 16
Train_image shape (512,512)

Test_image shape: (512,512)

Bounding box Vertices: ([[[473 144]]

 [[472 140]]

 [[473 144]]

 [[474 142]]])
Test_image: Task2Dataset/TestWithoutRotations/images/test_image_1.png

Train_image: 002-bike.png

16 16
Test_image: Task2Dataset/TestWithoutRotations/images/test_image_1.png

Train_image: 003-bridge-1.png

23 23
Test_image: Task2Dataset/TestWithoutRotations/images/test_image_1.png

Train_image: 004-bridge.png

20 20
Test_image: Task2Dataset/TestWithoutRotations/images/test_image_1.png

Train_image: 005-silo.png

16 16
Test_image: Task2Dataset/TestWithoutRotations/images/test_image_1.png

Train_image: 006-church.png

1 1
Test_image: Task2Dataset/TestWithoutRotations/images/test_image_1.png

Train_image: 007-supermarket.png

10 10
Train_image shape (512,512)

Test_image shape: (512,512)

Bounding box Vertices: ([[[457 182]]

 [[456 180]]

 [[4

In [12]:
# FLANN with RANSAC - shit
sift = SIFT(train_folder=task_2_train, 
            test_folder=task_2_test, 
            test_annotation_folder=task_2_test_annotations, 
            matcher=cv2.FlannBasedMatcher_create(),
            lowe_thresh=0.3, min_matches=3, n_feat=1000, n_octaves=5, c_thresh=0.1)

sift.get_annotations()
sift.get_sift_train_features()
sift.eval(normalise=True)

In [6]:
#bf with lowes
sift = SIFT(train_folder=task_2_train, 
            test_folder=task_2_test, 
            test_annotation_folder=task_2_test_annotations, 
            matcher=cv2.BFMatcher_create(),
            lowe_thresh=0.3, min_matches=3, n_feat=1000, n_octaves=5, c_thresh=0.1)

sift.get_annotations()
sift.get_sift_train_features()
sift.eval(normalise=True, lowes=True)

['gas-station', 'trash', 'theater', 'house']
{'gas': 0.17647058823529413, 'theater': 0.06060606060606061, 'house': 0.04, 'atm': 0.025}
-----------------------------
test_image_1.txt : 2 / 4
['factory', 'hotel', 'university', 'bank', 'cinema']
{'hotel': 0.13253012048192772, 'bank': 0.09090909090909091, 'factory': 0.031914893617021274, 'university': 0.021739130434782608, 'hospital': 0.0196078431372549}
-----------------------------
test_image_10.txt : 4 / 5
['supermarket', 'post-office', 'bridge', 'van']
{'van': 0.17543859649122806, 'supermarket': 0.16842105263157894, 'post': 0.09210526315789473, 'bridge': 0.06956521739130435}
-----------------------------
test_image_11.txt : 3 / 4
['flower', 'cemetery', 'traffic-light', 'fountain', 'ferris-wheel']
{'cemetery': 0.18181818181818182, 'ferris': 0.12962962962962962, 'traffic': 0.05172413793103448, 'flower': 0.04411764705882353, 'lighthouse': 0.0}
-----------------------------
test_image_12.txt : 2 / 5
['government', 'telephone-booth', 'car',

In [7]:
#bf with ransac
sift = SIFT(train_folder=task_2_train, 
            test_folder=task_2_test, 
            test_annotation_folder=task_2_test_annotations, 
            matcher=cv2.BFMatcher_create(),
            lowe_thresh=0.3, min_matches=3, n_feat=1000, n_octaves=5, c_thresh=0.1)

sift.get_annotations()
sift.get_sift_train_features()
sift.eval(normalise=True)

['gas-station', 'trash', 'theater', 'house']
{'trash': 0.49019607843137253, 'cemetery': 0.45454545454545453, 'church': 0.45, 'field': 0.4444444444444444}
-----------------------------
test_image_1.txt : 1 / 4
['factory', 'hotel', 'university', 'bank', 'cinema']
{'trash': 0.49019607843137253, 'cemetery': 0.45454545454545453, 'church': 0.4, 'field': 0.3888888888888889, 'prison': 0.38666666666666666}
-----------------------------
test_image_10.txt : 0 / 5
['supermarket', 'post-office', 'bridge', 'van']
{'trash': 0.6274509803921569, 'cemetery': 0.5454545454545454, 'van': 0.5087719298245614, 'church': 0.45}
-----------------------------
test_image_11.txt : 1 / 4
['flower', 'cemetery', 'traffic-light', 'fountain', 'ferris-wheel']
{'cemetery': 0.5454545454545454, 'traffic': 0.4482758620689655, 'billboard': 0.42696629213483145, 'museum': 0.4090909090909091, 'courthouse': 0.4}
-----------------------------
test_image_12.txt : 1 / 5
['government', 'telephone-booth', 'car', 'shop', 'cemetery']
{'

In [8]:
#flann with lowes
sift = SIFT(train_folder=task_2_train, 
            test_folder=task_2_test, 
            test_annotation_folder=task_2_test_annotations, 
            matcher=cv2.FlannBasedMatcher_create(),
            lowe_thresh=0.3, min_matches=3, n_feat=1000, n_octaves=5, c_thresh=0.1)

sift.get_annotations()
sift.get_sift_train_features()
sift.eval(normalise=True ,lowes=True)

['gas-station', 'trash', 'theater', 'house']
{'gas': 0.17647058823529413, 'theater': 0.06060606060606061, 'house': 0.05333333333333334, 'atm': 0.025}
-----------------------------
test_image_1.txt : 2 / 4
['factory', 'hotel', 'university', 'bank', 'cinema']
{'hotel': 0.13253012048192772, 'bank': 0.11363636363636363, 'factory': 0.031914893617021274, 'university': 0.021739130434782608, 'hospital': 0.0196078431372549}
-----------------------------
test_image_10.txt : 4 / 5
['supermarket', 'post-office', 'bridge', 'van']
{'supermarket': 0.17894736842105263, 'van': 0.17543859649122806, 'post': 0.09210526315789473, 'bridge': 0.06956521739130435}
-----------------------------
test_image_11.txt : 3 / 4
['flower', 'cemetery', 'traffic-light', 'fountain', 'ferris-wheel']
{'cemetery': 0.18181818181818182, 'ferris': 0.12962962962962962, 'traffic': 0.05172413793103448, 'flower': 0.04411764705882353, 'lighthouse': 0.0}
-----------------------------
test_image_12.txt : 2 / 5
['government', 'telephone

In [5]:
def get_sift_train_features(train_folder, nOctave, nFeat, contrast):
    
    train_points = {}
    train_descriptors = {}
    
    sift = cv2.SIFT_create(nOctaveLayers=nOctave, nfeatures=nFeat, contrastThreshold=contrast)

    for train_image in os.listdir(train_folder):
        image = cv2.imread(os.path.join(train_folder, train_image), 0)
        
        pt, desc = sift.detectAndCompute(image, None)
        # print(len(pt))
        train_points[train_image] = pt
        train_descriptors[train_image] = desc

    return train_descriptors, train_points

# get_sift_train_features(task_2_train, 3, 1000, 0.03)

In [6]:
def get_annotations(annotation_folder):
    annotations = {}

    for annotation_file in os.listdir(annotation_folder):
        image_name = os.path.basename(annotation_file)
        annotation_file = open(os.path.join(annotation_folder, annotation_file), "r")
        annotation_lines = annotation_file.readlines()
        image_annot = []
        for line in annotation_lines:
            image_annot.append(line.split(",")[0])
        annotations[image_name] = image_annot
    return annotations



In [30]:
def sift_matches(train_pt, train_desc, test_image_path, cv2Matcher, ratioThresh, nOctave, nFeat, contrastThresh, edgeThresh, sig):

    test_image = cv2.imread(test_image_path, 0)

    sift = cv2.SIFT_create(nOctaveLayers=nOctave, nfeatures=nFeat, contrastThreshold=contrastThresh, edgeThreshold=edgeThresh, sigma=sig)

    test_points, test_descriptor = sift.detectAndCompute(test_image, None)
    
    matches = cv2Matcher.knnMatch(train_desc, test_descriptor, 2)
    retained_matches = []
    min_matches=3
    for one, two in matches:
        #change threshold; lower to reduce matches retained
        if one.distance < ratioThresh*two.distance:
            retained_matches.append([one])
    
    if len(retained_matches) > min_matches:
        print(f"Retained matches: {len(retained_matches)}")
        src_pts = np.float32([train_pt[m[0].queryIdx].pt for m in retained_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([test_points[m[0].trainIdx].pt for m in retained_matches]).reshape(-1, 1, 2)
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        
        # print("Homography matrix:\n", H)
        return H, mask
    else:
        #print("Not enough matches found")

        return None, None
        print("Not enough matches found")
 
    return retained_matches
 

In [44]:
def evaluate_sift(train_folder, test_image_folder, test_annotation_folder):
    total_total = 0
    total_correct = 0
    for test_image in os.listdir(test_image_folder):
        
        train_pts, train_desc = get_sift_train_features(train_folder, nOctave=5, nFeat=1000, contrast=0.1)
        
        scores = {}
        
        for image, desc in train_desc.items():
            
            
            matcher = cv2.BFMatcher_create()#set crossCheck==true as alternative to ratio test
            # matches = sift_matches(train_desc[image], train_pts[image], os.path.join(test_image_folder, test_image), matcher, ratioThresh=0.3, nOctave=3, nFeat=0, contrastThresh=0.04, edgeThresh=10, sig=1.6)
            H,mask = sift_matches(train_desc[image], train_pts[image], os.path.join(test_image_folder, test_image), matcher, ratioThresh=0.75, nOctave=3, nFeat=10, contrastThresh=0.04, edgeThresh=10, sig=1.6)
            if H is None:
                scores[image.split("-")[1].split(".")[0]] = 0
            else:
                canvas = cv2.imread(test_image_folder + test_image)
                matches_mask = mask.ravel().tolist()
                inliers = [bool(val) for val in matches_mask]

                h, w = canvas.shape[:2]
                pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
                print(f"H : {H}")
                dst = cv2.perspectiveTransform(pts, H)
                draw_params = dict(matchColor = (0,255,0),
                                   singlePointColor = None,
                                   matchesMask = matches_mask,
                                   flags=2)
                
                print(f"DST: {dst}")
                # Draw the matched features and bounding box
                result = cv2.polylines(canvas, [np.int32(dst)], True, (0,0,255),2, cv2.LINE_AA)
                # result = cv2.drawMatches(image, kp1, img2, kp2, good, inliers)
                cv2.imshow("result", result)
                cv2.waitKey()
                break
                scores[image.split("-")[1].split(".")[0]] = len(inliers)
        
            # scores[image.split("-")[1].split(".")[0]] = len(matches)

    #     scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))
    #     annotations = get_annotations(test_annotation_folder)

    #     correct, total = get_metrics(annotations, scores, test_image)
    #     total_total+=total
    #     total_correct+=correct
    # print("Total performance:", total_correct, "/", total_total)
    # print(total_correct/total_total)
                


In [45]:
evaluate_sift(task_2_train, task_2_test, task_2_test_annotations)

Retained matches: 4
(3, 3)
H : [[ nan  nan  nan]
 [ inf -inf -inf]
 [ nan  nan  nan]]
DST: [[[0. 0.]]

 [[0. 0.]]

 [[0. 0.]]

 [[0. 0.]]]
Retained matches: 6


AttributeError: 'NoneType' object has no attribute 'shape'

In [14]:
evaluate_sift(task_2_train, task_3_test, task_3_test_annotations)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Task3AdditionalTestDataset/images/'

In [ ]:
##old github post
import matplotlib, cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import math
import os
import itertools
import random
%matplotlib inline
task_2_train = "Task2Dataset/Training/png/"
task_2_test = "Task2Dataset/TestWithoutRotations/images/"
task_2_test_annotations = "Task2Dataset/TestWithoutRotations/annotations/"

task_3_test_annotations = "Task3AdditionalTestDataset/annotations/"
task_3_test = "Task3AdditionalTestDataset/images/"

def get_train_features(train_folder):
    
    train_points = {}
    train_descriptors = {}
    
    sift = cv2.SIFT_create(nOctaveLayers=5, nfeatures=1000, contrastThreshold=0.1)

    for train_image in os.listdir(train_folder):
        image = cv2.imread(os.path.join(train_folder, train_image), 0)
        
        pt, desc = sift.detectAndCompute(image, None)
        
        train_points[train_image] = pt
        train_descriptors[train_image] = desc

    return train_descriptors, train_points

def get_annotations(annotation_folder):
    annotations = {}

    for annotation_file in os.listdir(annotation_folder):
        image_name = os.path.basename(annotation_file)
        annotation_file = open(os.path.join(annotation_folder, annotation_file), "r")
        annotation_lines = annotation_file.readlines()
        image_annot = []
        for line in annotation_lines:
            image_annot.append(line.split(",")[0])
        annotations[image_name] = image_annot
    return annotations

def sift_matches(train_pt, train_desc, test_image_path):

    test_image = cv2.imread(test_image_path, 0)

    sift = cv2.SIFT_create(nOctaveLayers=5, nfeatures=1000, contrastThreshold=0.1)

    test_points, test_descriptor = sift.detectAndCompute(test_image, None)

    matcher = cv2.BFMatcher_create()

    matches = matcher.knnMatch(train_desc, test_descriptor, 2)
    retained_matches = []
    for one, two in matches:
        #change threshold; lower to reduce matches retained
        if one.distance < 0.3*two.distance:
            retained_matches.append([one])

    return retained_matches
 
def evaluate(train_folder, test_image_folder, test_annotation_folder):
    total_total = 0
    total_correct = 0
    for test_image in os.listdir(test_image_folder):
        
        train_pts, train_desc = get_train_features(train_folder)
        
        scores = {}
        
        for image, desc in train_desc.items():

            matches = sift_matches(train_desc[image], train_pts[image], os.path.join(test_image_folder, test_image))
            scores[image.split("-")[1].split(".")[0]] = len(matches)

        scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))
        annotations = get_annotations(test_annotation_folder)

        scores = dict(itertools.islice(scores.items(), len(annotations)))
        print(scores)
        # note - need to refactor task 3 test annotations from .csv to .txt before use
        test_annotations = annotations[test_image.split(".")[0]+".txt"]
        true_positives, false_positives, true_negatives, false_negatives = 0,0,0,0 
        for annotation in test_annotations:
            if annotation in scores:
                true_positives += 1
            else:
                false_positives += 1
                false_negatives += 1
        total = false_positives + true_positives
        total_total+=total
        total_correct+=true_positives
        print(test_image, ":", true_positives, "/", total)
    print("Total performance:", total_correct, "/", total_total)
                


evaluate(task_2_train, task_2_test, task_2_test_annotations)
